In [ ]:
DEBUG = False
TRAIN = False


MAX_SEQ_LENGTH = 128

FEATURE_NAMES = [
    'acc_x', 'acc_y', 'acc_z',
    'rot_w', 'rot_x', 'rot_y', 'rot_z',
    'acc_mag', 'rot_angle', 'acc_mag_jerk', 'rot_angle_vel',
    'linear_acc_x', 'linear_acc_y', 'linear_acc_z', 'linear_acc_mag', 'linear_acc_mag_jerk',
    'angular_vel_x', 'angular_vel_y', 'angular_vel_z',
    'angular_distance',
]
CATEGORICAL_FEATURES = []
NUMERICAL_FEATURES = [f for f in FEATURE_NAMES if f not in CATEGORICAL_FEATURES]

LABEL_NAMES = [
    'Forehead - pull hairline',
    'Neck - pinch skin',
    'Forehead - scratch',
    'Eyelash - pull hair',
    'Text on phone',
    'Eyebrow - pull hair',
    'Neck - scratch',
    'Above ear - pull hair',
    'Cheek - pinch skin',
    'Wave hello',
    'Write name in air',
    'Pull air toward your face',
    'Feel around in tray and pull out an object',
    'Write name on leg',
    'Pinch knee/leg skin',
    'Scratch knee/leg skin',
    'Drink from bottle/cup',
    'Glasses on/off'
]
IDX2LABEL = {x: i for i, x in enumerate(LABEL_NAMES)}

OUTPUT_DIR = './saved_models'

In [ ]:
import os
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

## Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib

In [ ]:
DATA_ROOT = '/kaggle/input/cmi-detect-behavior-with-sensor-data/'

train_df = pd.read_csv(f'{DATA_ROOT}/train.csv')
train_demo_df = pd.read_csv(f'{DATA_ROOT}/train_demographics.csv')

In [ ]:
train_df = pd.merge(
    train_df,
    train_demo_df,
    how='left',
    on='subject'
)

train_df.shape[0]

## Feature Engineering

In [ ]:
# Remove Gravity

from scipy.spatial.transform import Rotation as R

def remove_gravity_from_acc(acc_data, rot_data):

    if isinstance(acc_data, pd.DataFrame):
        acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values
    else:
        acc_values = acc_data

    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = acc_values.shape[0]
    linear_accel = np.zeros_like(acc_values)
    
    gravity_world = np.array([0, 0, 9.81])

    for i in range(num_samples):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i, :] = acc_values[i, :] 
            continue

        try:
            rotation = R.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
             linear_accel[i, :] = acc_values[i, :]
             
    return linear_accel

def calculate_angular_velocity_from_quat(rot_data, time_delta=1/200): # Assuming 200Hz sampling rate
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_vel = np.zeros((num_samples, 3))

    for i in range(num_samples - 1):
        q_t = quat_values[i]
        q_t_plus_dt = quat_values[i+1]

        if np.all(np.isnan(q_t)) or np.all(np.isclose(q_t, 0)) or \
           np.all(np.isnan(q_t_plus_dt)) or np.all(np.isclose(q_t_plus_dt, 0)):
            continue

        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)

            # Calculate the relative rotation
            delta_rot = rot_t.inv() * rot_t_plus_dt
            
            # Convert delta rotation to angular velocity vector
            # The rotation vector (Euler axis * angle) scaled by 1/dt
            # is a good approximation for small delta_rot
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError:
            # If quaternion is invalid, angular velocity remains zero
            pass
            
    return angular_vel

def calculate_angular_distance(rot_data):
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_dist = np.zeros(num_samples)

    for i in range(num_samples - 1):
        q1 = quat_values[i]
        q2 = quat_values[i+1]

        if np.all(np.isnan(q1)) or np.all(np.isclose(q1, 0)) or \
           np.all(np.isnan(q2)) or np.all(np.isclose(q2, 0)):
            angular_dist[i] = 0 # Или np.nan, в зависимости от желаемого поведения
            continue
        try:
            # Преобразование кватернионов в объекты Rotation
            r1 = R.from_quat(q1)
            r2 = R.from_quat(q2)

            # Вычисление углового расстояния: 2 * arccos(|real(p * q*)|)
            # где p* - сопряженный кватернион q
            # В scipy.spatial.transform.Rotation, r1.inv() * r2 дает относительное вращение.
            # Угол этого относительного вращения - это и есть угловое расстояние.
            relative_rotation = r1.inv() * r2
            
            # Угол rotation vector соответствует угловому расстоянию
            # Норма rotation vector - это угол в радианах
            angle = np.linalg.norm(relative_rotation.as_rotvec())
            angular_dist[i] = angle
        except ValueError:
            angular_dist[i] = 0 # В случае недействительных кватернионов
            pass
            
    return angular_dist

In [ ]:
def feature_engineering(train_df):
    # IMU magnitude
    train_df['acc_mag'] = np.sqrt(train_df['acc_x']**2 + train_df['acc_y']**2 + train_df['acc_z']**2)
    
    # IMU angle
    train_df['rot_angle'] = 2 * np.arccos(train_df['rot_w'].clip(-1, 1))
    
    # IMU jerk, angular velocity
    train_df['acc_mag_jerk'] = train_df.groupby('sequence_id')['acc_mag'].diff().fillna(0)
    train_df['rot_angle_vel'] = train_df.groupby('sequence_id')['rot_angle'].diff().fillna(0)
    
    # Remove gravity
    def get_linear_accel(df):
        res = remove_gravity_from_acc(
            df[['acc_x', 'acc_y', 'acc_z']],
            df[['rot_x', 'rot_y', 'rot_z', 'rot_w']]
        )
        res = pd.DataFrame(res, columns=['linear_acc_x', 'linear_acc_y', 'linear_acc_z'], index=df.index)
        return res
    
    linear_accel_df = train_df.groupby('sequence_id').apply(get_linear_accel, include_groups=False)
    linear_accel_df = linear_accel_df.droplevel('sequence_id')
    train_df = train_df.join(linear_accel_df)
    
    train_df['linear_acc_mag'] = np.sqrt(train_df['linear_acc_x']**2 + train_df['linear_acc_y']**2 + train_df['linear_acc_z']**2)
    train_df['linear_acc_mag_jerk'] = train_df.groupby('sequence_id')['linear_acc_mag'].diff().fillna(0)

    # Calc angular velocity
    def calc_angular_velocity(df):
        res = calculate_angular_velocity_from_quat( df[['rot_x', 'rot_y', 'rot_z', 'rot_w']] )
        res = pd.DataFrame(res, columns=['angular_vel_x', 'angular_vel_y', 'angular_vel_z'], index=df.index)
        return res
    
    angular_velocity_df = train_df.groupby('sequence_id').apply(calc_angular_velocity, include_groups=False)
    angular_velocity_df = angular_velocity_df.droplevel('sequence_id')
    train_df = train_df.join(angular_velocity_df)

    # Calculating angular distance
    def calc_angular_distance(df):
        res = calculate_angular_distance(df[['rot_x', 'rot_y', 'rot_z', 'rot_w']])
        res = pd.DataFrame(res, columns=['angular_distance'], index=df.index)
        return res
    
    angular_distance_df = train_df.groupby('sequence_id').apply(calc_angular_distance, include_groups=False)
    angular_distance_df = angular_distance_df.droplevel('sequence_id')
    train_df = train_df.join(angular_distance_df)

    train_df[FEATURE_NAMES] = train_df[FEATURE_NAMES].ffill().bfill().fillna(0).values.astype('float32')
    
    return train_df

In [ ]:
train_df = feature_engineering(train_df)

### Standard

In [ ]:
feature_scaler = StandardScaler()
feature_scaler.fit(train_df[NUMERICAL_FEATURES])
joblib.dump(feature_scaler, f'{OUTPUT_DIR}/feature_scaler.joblib')

In [ ]:
train_df[NUMERICAL_FEATURES] = feature_scaler.transform(train_df[NUMERICAL_FEATURES])

In [ ]:
len(NUMERICAL_FEATURES)

### Convert to Sequence

In [ ]:
agg_train_df = train_df.groupby(['sequence_id', 'subject', 'gesture']).apply(
    lambda df: df[FEATURE_NAMES].to_dict(orient='records'),
    include_groups=False,
)

agg_train_df = agg_train_df.reset_index()
agg_train_df.columns = ['sequence_id', 'subject', 'gesture', 'sequence']

In [ ]:
agg_train_df['label'] = agg_train_df.gesture.map(IDX2LABEL)

In [ ]:
agg_train_df.head()

## Data Augmentation

In [ ]:
import torch
import numpy as np
import random
from typing import Dict, List, Optional, Tuple
from scipy import signal
from scipy.interpolate import interp1d


class TimeSeriesAugmentation:
    """时序数据增强类，专门用于IMU和ToF传感器数据"""
    
    def __init__(self, 
                 time_stretch_range: Tuple[float, float] = (0.8, 1.2),
                 time_shift_range: float = 0.1,
                 noise_std: float = 0.02,
                 magnitude_scale_range: Tuple[float, float] = (0.9, 1.1),
                 rotation_angle_range: float = 0.1,
                 mask_ratio: float = 0.1,
                 freq_filter_range: Tuple[float, float] = (0.1, 0.9),
                 aug_prob: float = 0.8):
        """
        初始化数据增强参数
        
        Args:
            time_stretch_range: 时间拉伸范围 (最小比例, 最大比例)
            time_shift_range: 时间移位范围 (序列长度的比例)
            noise_std: 高斯噪声标准差
            magnitude_scale_range: 幅度缩放范围
            rotation_angle_range: 旋转角度范围 (弧度)
            mask_ratio: 时间遮盖比例
            freq_filter_range: 频率滤波范围
            aug_prob: 每种增强方法的应用概率
        """
        self.time_stretch_range = time_stretch_range
        self.time_shift_range = time_shift_range
        self.noise_std = noise_std
        self.magnitude_scale_range = magnitude_scale_range
        self.rotation_angle_range = rotation_angle_range
        self.mask_ratio = mask_ratio
        self.freq_filter_range = freq_filter_range
        self.aug_prob = aug_prob
        
        # IMU特征索引 (前7个特征: acc_x, acc_y, acc_z, rot_w, rot_x, rot_y, rot_z)
        self.imu_indices = list(range(7))
        self.acc_indices = [0, 1, 2]  # 加速度
        self.rot_indices = [3, 4, 5, 6]  # 四元数旋转
    
    def __call__(self, sequence: np.ndarray) -> np.ndarray:
        """
        对序列应用随机增强
        
        Args:
            sequence: 输入序列 (seq_len, num_features)
        
        Returns:
            augmented_sequence: 增强后的序列
        """
        augmented = sequence.copy()
        
        # 随机应用各种增强方法
        if random.random() < self.aug_prob:
            augmented = self.time_stretch(augmented)
        
        if random.random() < self.aug_prob:
            augmented = self.time_shift(augmented)
        
        if random.random() < self.aug_prob:
            augmented = self.add_noise(augmented)
        
        if random.random() < self.aug_prob:
            augmented = self.magnitude_scale(augmented)
        
        if random.random() < self.aug_prob:
            augmented = self.rotate_imu(augmented)
        
        if random.random() < self.aug_prob:
            augmented = self.time_mask(augmented)
        
        if random.random() < self.aug_prob:
            augmented = self.frequency_filter(augmented)
        
        return augmented
    
    def time_stretch(self, sequence: np.ndarray) -> np.ndarray:
        """时间拉伸/压缩"""
        seq_len, num_features = sequence.shape
        stretch_factor = random.uniform(*self.time_stretch_range)
        
        # 创建新的时间采样点
        original_time = np.linspace(0, 1, seq_len)
        new_length = max(1, int(seq_len / stretch_factor))
        new_time = np.linspace(0, 1, new_length)
        
        # 对每个特征进行插值
        stretched_sequence = np.zeros((new_length, num_features))
        for i in range(num_features):
            interpolator = interp1d(original_time, sequence[:, i], 
                                  kind='linear', bounds_error=False, 
                                  fill_value='extrapolate')
            stretched_sequence[:, i] = interpolator(new_time)
        
        # 如果序列变长，截断到原长度；如果变短，用零填充
        if new_length > seq_len:
            return stretched_sequence[:seq_len]
        else:
            padded = np.zeros((seq_len, num_features))
            padded[:new_length] = stretched_sequence
            return padded
    
    def time_shift(self, sequence: np.ndarray) -> np.ndarray:
        """时间移位"""
        seq_len, num_features = sequence.shape
        max_shift = int(seq_len * self.time_shift_range)
        shift = random.randint(-max_shift, max_shift)
        
        if shift == 0:
            return sequence
        
        shifted = np.zeros_like(sequence)
        if shift > 0:
            shifted[shift:] = sequence[:-shift]
        else:
            shifted[:shift] = sequence[-shift:]
        
        return shifted
    
    def add_noise(self, sequence: np.ndarray) -> np.ndarray:
        """添加高斯噪声"""
        noise = np.random.normal(0, self.noise_std, sequence.shape)
        return sequence + noise
    
    def magnitude_scale(self, sequence: np.ndarray) -> np.ndarray:
        """幅度缩放"""
        scale_factor = random.uniform(*self.magnitude_scale_range)
        scaled = sequence.copy()
        
        # 只对IMU数据进行缩放，保持其他特征不变
        scaled[:, self.imu_indices] *= scale_factor
        
        return scaled
    
    def rotate_imu(self, sequence: np.ndarray) -> np.ndarray:
        """对IMU数据应用小角度旋转"""
        rotated = sequence.copy()
        
        # 生成小角度旋转矩阵
        angle = random.uniform(-self.rotation_angle_range, self.rotation_angle_range)
        cos_a, sin_a = np.cos(angle), np.sin(angle)
        
        # 只旋转加速度数据 (简化版本，只在xy平面旋转)
        acc_x = rotated[:, 0] * cos_a - rotated[:, 1] * sin_a
        acc_y = rotated[:, 0] * sin_a + rotated[:, 1] * cos_a
        
        rotated[:, 0] = acc_x
        rotated[:, 1] = acc_y
        
        return rotated
    
    def time_mask(self, sequence: np.ndarray) -> np.ndarray:
        """时间遮盖 - 随机遮盖一段时间窗口"""
        seq_len, num_features = sequence.shape
        mask_length = int(seq_len * self.mask_ratio)
        
        if mask_length == 0:
            return sequence
        
        start_idx = random.randint(0, seq_len - mask_length)
        masked = sequence.copy()
        
        # 用前后平均值填充遮盖区域
        if start_idx > 0 and start_idx + mask_length < seq_len:
            before_mean = np.mean(masked[max(0, start_idx-5):start_idx], axis=0)
            after_mean = np.mean(masked[start_idx+mask_length:min(seq_len, start_idx+mask_length+5)], axis=0)
            fill_value = (before_mean + after_mean) / 2
        else:
            fill_value = np.mean(masked, axis=0)
        
        masked[start_idx:start_idx+mask_length] = fill_value
        
        return masked
    
    def frequency_filter(self, sequence: np.ndarray) -> np.ndarray:
        """频域滤波"""
        seq_len, num_features = sequence.shape
        if seq_len < 10:  # 序列太短，跳过滤波
            return sequence
        
        filtered = sequence.copy()
        
        # 只对IMU数据进行滤波
        for i in self.imu_indices:
            # 应用低通滤波或高通滤波
            if random.random() < 0.5:  # 低通滤波
                cutoff = random.uniform(*self.freq_filter_range)
                sos = signal.butter(2, cutoff, btype='low', output='sos')
            else:  # 高通滤波
                cutoff = random.uniform(0.05, self.freq_filter_range[0])
                sos = signal.butter(2, cutoff, btype='high', output='sos')
            
            try:
                filtered[:, i] = signal.sosfilt(sos, sequence[:, i])
            except:
                # 如果滤波失败，保持原数据
                pass
        
        return filtered


class MixupAugmentation:
    """Mixup数据增强 - 混合两个不同的序列"""
    
    def __init__(self, alpha: float = 0.2, prob: float = 0.5):
        """
        Args:
            alpha: Beta分布参数
            prob: 应用Mixup的概率
        """
        self.alpha = alpha
        self.prob = prob
    
    def __call__(self, batch_sequences: torch.Tensor, batch_labels: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, float]:
        """
        对批次数据应用Mixup
        
        Args:
            batch_sequences: (batch_size, seq_len, num_features)
            batch_labels: (batch_size,)
        
        Returns:
            mixed_sequences: 混合后的序列
            labels_a: 原始标签
            labels_b: 混合标签
            lambda_: 混合比例
        """
        if random.random() > self.prob:
            return batch_sequences, batch_labels, batch_labels, 1.0
        
        batch_size = batch_sequences.size(0)
        lambda_ = np.random.beta(self.alpha, self.alpha) if self.alpha > 0 else 1
        
        # 随机打乱顺序
        index = torch.randperm(batch_size)
        
        # 混合序列
        mixed_sequences = lambda_ * batch_sequences + (1 - lambda_) * batch_sequences[index]
        
        labels_a = batch_labels
        labels_b = batch_labels[index]
        
        return mixed_sequences, labels_a, labels_b, lambda_


def mixup_criterion(criterion, outputs, labels_a, labels_b, lambda_):
    """Mixup损失函数"""
    return lambda_ * criterion(outputs, labels_a) + (1 - lambda_) * criterion(outputs, labels_b)

## DataLoader

In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold
import pandas as pd
from typing import List, Dict

class SequenceDataset(Dataset):
    def __init__(self, sequences: List[List[Dict]], labels: List[int], is_training=True, use_augmentation=True, augmentation_params=None):
        """
        Args:
            sequences: 序列数据列表，每个序列包含多个时间步的特征字典
            labels: 对应的标签列表
        """
        self.sequences = sequences
        self.labels = labels
        self.is_training = is_training
        self.use_augmentation = use_augmentation and is_training
        
        # 转换为numpy数组格式并截断
        self.processed_sequences = self._process_sequences()

        # 初始化数据增强器
        if self.use_augmentation:
            if augmentation_params is None:
                augmentation_params = {
                    'time_stretch_range': (0.8, 1.2),
                    'time_shift_range': 0.1,
                    'noise_std': 0.02,
                    'magnitude_scale_range': (0.9, 1.1),
                    'rotation_angle_range': 0.1,
                    'mask_ratio': 0.1,
                    'freq_filter_range': (0.1, 0.9),
                    'aug_prob': 0.5  # 训练时降低概率，避免过度增强
                }
            self.augmenter = TimeSeriesAugmentation(**augmentation_params)
            print(f"✅ 数据增强已启用 (训练模式: {self.is_training})")
        else:
            self.augmenter = None
            print(f"❌ 数据增强已禁用 (训练模式: {self.is_training})")
        
        
    def _process_sequences(self):
        """将字典格式的序列转换为numpy数组并截断到最大长度"""
        processed = []
        
        for seq in self.sequences:
            # 截断到最大长度
            seq = seq[:MAX_SEQ_LENGTH]
            
            seq_array = []
            for timestep in seq:
                # 按照feature_names的顺序提取特征
                features = [timestep[feature] for feature in FEATURE_NAMES]
                seq_array.append(features)
            processed.append(np.array(seq_array, dtype=np.float32))
        
        return processed
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        sequence = self.processed_sequences[idx].copy()
        label = torch.LongTensor([self.labels[idx]])[0]

        # 在训练时应用数据增强
        if self.use_augmentation and self.augmenter is not None:
            sequence = self.augmenter(sequence)
        
        sequence = torch.FloatTensor(sequence)
        return sequence, label


def collate_fn(batch):
    """自定义批处理函数，处理不同长度的序列"""
    sequences, labels = zip(*batch)
    
    # 获取批次中的最大序列长度
    max_length = max(seq.shape[0] for seq in sequences)
    
    # 填充序列到相同长度
    padded_sequences = []
    lengths = []
    
    for seq in sequences:
        seq_len = seq.shape[0]
        lengths.append(seq_len)
        
        if seq_len < max_length:
            # 使用零填充
            padding = torch.zeros(max_length - seq_len, seq.shape[1])
            padded_seq = torch.cat([seq, padding], dim=0)
        else:
            padded_seq = seq
        
        padded_sequences.append(padded_seq)
    
    return torch.stack(padded_sequences), torch.stack(labels), torch.LongTensor(lengths)


def create_kfold_splits(sequences, labels, subjects, n_splits=5, random_state=42):
    """使用StratifiedGroupKFold创建K折分割"""
    sequences = np.array(sequences, dtype=object)
    labels = np.array(labels)
    subjects = np.array(subjects)

    if sequences.shape[0] > 1000:
        sgkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    else:
        n_splits = 2
        sgkf = GroupKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    splits = []
    for fold_idx, (train_idx, val_idx) in enumerate(sgkf.split(sequences, labels, subjects)):
        splits.append((train_idx, val_idx))
        print(f"   Fold {fold_idx+1}: 训练={len(train_idx)}, 验证={len(val_idx)}")
    
    print(f"✅ 成功创建{n_splits}折交叉验证分割")
    return splits


def create_fold_loaders(sequences, labels, train_idx, val_idx, batch_size=32, use_augmentation=True, augmentation_params=None, use_mixup=False, mixup_params=None):
    """为特定的折创建数据加载器，使用全局scaler"""
    # 根据索引分割数据
    train_sequences = [sequences[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_sequences = [sequences[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]
    
    # 创建数据集，使用全局scaler
    train_dataset = SequenceDataset(train_sequences, train_labels, is_training=True, use_augmentation=use_augmentation, augmentation_params=augmentation_params)
    val_dataset = SequenceDataset(val_sequences, val_labels, is_training=False, use_augmentation=False)
    
    # 创建数据加载器
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # 创建Mixup增强器
    mixup_augmenter = None
    if use_mixup:
        if mixup_params is None:
            mixup_params = {'alpha': 0.2, 'prob': 0.5}
        mixup_augmenter = MixupAugmentation(**mixup_params)
        print(f"✅ Mixup数据增强已启用: alpha={mixup_params['alpha']}, prob={mixup_params['prob']}")
    
    return train_loader, val_loader, mixup_augmenter

## Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=8):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool1d(1)
        self.excitation = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        b, c, _ = x.size()
        y = self.squeeze(x).view(b, c)
        y = self.excitation(y).view(b, c, 1)
        return x * y.expand_as(x)

class ResidualSECNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, pool_size=2, dropout=0.3, weight_decay=1e-4):
        super().__init__()
        
        # First conv block
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=kernel_size//2, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        
        # Second conv block
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=kernel_size//2, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        # SE block
        self.se = SEBlock(out_channels)
        
        # Shortcut connection
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, 1, bias=False),
                nn.BatchNorm1d(out_channels)
            )
        
        self.pool = nn.MaxPool1d(pool_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        shortcut = self.shortcut(x)
        
        # First conv
        out = F.relu(self.bn1(self.conv1(x)))
        # Second conv
        out = self.bn2(self.conv2(out))
        
        # SE block
        out = self.se(out)
        
        # Add shortcut
        out += shortcut
        out = F.relu(out)
        
        # Pool and dropout
        out = self.pool(out)
        out = self.dropout(out)
        
        return out

class AttentionLayer(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attention = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        # x shape: (batch, seq_len, hidden_dim)
        scores = torch.tanh(self.attention(x))  # (batch, seq_len, 1)
        weights = F.softmax(scores.squeeze(-1), dim=1)  # (batch, seq_len)
        context = torch.sum(x * weights.unsqueeze(-1), dim=1)  # (batch, hidden_dim)
        return context

class TwoBranchModel(nn.Module):
    def __init__(self, imu_dim, tof_dim, n_classes, weight_decay=1e-4):
        super().__init__()
        self.imu_dim = imu_dim
        self.tof_dim = tof_dim
        self.n_classes = n_classes
        self.weight_decay = weight_decay
        
        # IMU deep branch
        self.imu_block1 = ResidualSECNNBlock(imu_dim, 64, 3, dropout=0.3, weight_decay=weight_decay)
        self.imu_block2 = ResidualSECNNBlock(64, 128, 5, dropout=0.3, weight_decay=weight_decay)
        
        # # TOF/Thermal lighter branch
        # self.tof_conv1 = nn.Conv1d(tof_dim, 64, 3, padding=1, bias=False)
        # self.tof_bn1 = nn.BatchNorm1d(64)
        # self.tof_pool1 = nn.MaxPool1d(2)
        # self.tof_drop1 = nn.Dropout(0.3)
        
        # self.tof_conv2 = nn.Conv1d(64, 128, 3, padding=1, bias=False)
        # self.tof_bn2 = nn.BatchNorm1d(128)
        # self.tof_pool2 = nn.MaxPool1d(2)
        # self.tof_drop2 = nn.Dropout(0.3)
        
        # BiLSTM
        # self.bilstm = nn.LSTM(256, 128, bidirectional=True, batch_first=True)
        # self.bilstm = nn.LSTM(128, 128, bidirectional=True, batch_first=True)
        # self.lstm_dropout = nn.Dropout(0.4)
        self.bigru = nn.GRU(128, 128, bidirectional=True, batch_first=True)
        self.gru_dropout = nn.Dropout(0.4)
        
        # Attention
        self.attention = AttentionLayer(256)  # 128*2 for bidirectional
        
        # Dense layers
        self.dense1 = nn.Linear(256, 256, bias=False)
        self.bn_dense1 = nn.BatchNorm1d(256)
        self.drop1 = nn.Dropout(0.5)
        
        self.dense2 = nn.Linear(256, 128, bias=False)
        self.bn_dense2 = nn.BatchNorm1d(128)
        self.drop2 = nn.Dropout(0.3)
        
        self.classifier = nn.Linear(128, n_classes)
        
    def forward(self, x):
        # Split input
        # imu = x[:, :, :self.imu_dim].transpose(1, 2)  # (batch, imu_dim, seq_len)
        # tof = x[:, :, self.imu_dim:].transpose(1, 2)  # (batch, tof_dim, seq_len)

        imu = x[:, :, :self.imu_dim]  # imu only   batch, seq_len, hidden_dim
        imu = imu.transpose(1, 2)  # (batch, imu_dim, seq_len)

        
        # IMU branch
        x1 = self.imu_block1(imu)
        x1 = self.imu_block2(x1)
        
        # TOF branch
        # x2 = F.relu(self.tof_bn1(self.tof_conv1(tof)))
        # x2 = self.tof_drop1(self.tof_pool1(x2))
        # x2 = F.relu(self.tof_bn2(self.tof_conv2(x2)))
        # x2 = self.tof_drop2(self.tof_pool2(x2))
        
        # Concatenate branches
        # merged = torch.cat([x1, x2], dim=1).transpose(1, 2)  # (batch, seq_len, 256)
        merged = x1.transpose(1, 2)  # (batch, seq_len, 128)
        
        # BiLSTM
        # lstm_out, _ = self.bilstm(merged)
        # lstm_out = self.lstm_dropout(lstm_out)
        gru_out, _ = self.bigru(merged)
        gru_out = self.gru_dropout(gru_out)
        
        # Attention
        attended = self.attention(gru_out)
        
        # Dense layers
        x = F.relu(self.bn_dense1(self.dense1(attended)))
        x = self.drop1(x)
        x = F.relu(self.bn_dense2(self.dense2(x)))
        x = self.drop2(x)
        
        # Classification
        logits = self.classifier(x)
        return logits

## Train

### Metric

In [ ]:
import kagglehub
metric = kagglehub.package_import('jiazhuang/cmi-2025-metric')

def get_competition_score(true, pred):
    assert len(true) == len(pred)
    N = len(true)
    true = pd.DataFrame({'id': range(N), 'gesture': true})
    pred = pd.DataFrame({'id': range(N), 'gesture': pred})
    return metric.score(true, pred, 'id')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import json

# from metric import get_competition_score


class LabelSmoothingCrossEntropy(nn.Module):
    """Label Smoothing交叉熵损失函数"""
    
    def __init__(self, smoothing: float = 0.1, num_classes: int = None):
        """
        Args:
            smoothing: 平滑参数，范围[0, 1]，0表示不平滑，1表示均匀分布
            num_classes: 类别数量
        """
        super().__init__()
        self.smoothing = smoothing
        self.num_classes = num_classes
        
    def forward(self, pred, target):
        """
        Args:
            pred: 模型预测logits，形状[batch_size, num_classes]
            target: 真实标签，形状[batch_size]
        """
        batch_size, num_classes = pred.shape
        if self.num_classes is None:
            self.num_classes = num_classes
            
        # 计算log softmax
        log_pred = F.log_softmax(pred, dim=1)
        
        # 创建平滑标签
        with torch.no_grad():
            # 创建one-hot标签
            true_dist = torch.zeros_like(log_pred)
            true_dist.scatter_(1, target.unsqueeze(1), 1.0)
            
            # 应用label smoothing
            # 正确类别的概率 = 1 - smoothing + smoothing / num_classes
            # 其他类别的概率 = smoothing / num_classes
            true_dist = true_dist * (1 - self.smoothing) + self.smoothing / num_classes
        
        # 计算KL散度损失
        return torch.mean(torch.sum(-true_dist * log_pred, dim=1))


class Trainer:
    """LSTM分类器训练器"""
    
    def __init__(self, model, device, class_names, label_smoothing=0.0):
        self.model = model.to(device)
        self.device = device
        self.class_names = class_names
        self.label_smoothing = label_smoothing
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
        self.val_scores = []

        # 根据label_smoothing参数选择损失函数
        if label_smoothing > 0:
            self.criterion = LabelSmoothingCrossEntropy(
                smoothing=label_smoothing, 
                num_classes=len(class_names)
            )
            print(f"✅ 使用Label Smoothing (α={label_smoothing:.3f})")
        else:
            self.criterion = nn.CrossEntropyLoss()
            print("✅ 使用标准交叉熵损失")
        
    def train_epoch(self, train_loader, optimizer, criterion, mixup_augmenter=None):
        """训练一个epoch"""
        self.model.train()
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0
        
        progress_bar = tqdm(train_loader, desc="训练中", disable=True)
        
        for batch_idx, (sequences, labels, lengths) in enumerate(progress_bar):
            sequences = sequences.to(self.device)
            labels = labels.to(self.device)

            # 应用Mixup增强
            if mixup_augmenter is not None:
                sequences, labels_a, labels_b, lambda_ = mixup_augmenter(sequences, labels)
            else:
                labels_a, labels_b, lambda_ = labels, labels, 1.0
            
            # 前向传播
            optimizer.zero_grad()
            outputs = self.model(sequences)

            # 计算损失
            if mixup_augmenter is not None and lambda_ != 1.0:
                loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lambda_)
            else:
                loss = criterion(outputs, labels)
            
            # 反向传播
            loss.backward()
            
            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            # 统计
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)

            if mixup_augmenter is not None and lambda_ != 1.0:
                # Mixup时使用lambda_加权的准确率
                correct_predictions += (lambda_ * (predicted == labels_a).sum().item() + 
                                     (1 - lambda_) * (predicted == labels_b).sum().item())
            else:
                correct_predictions += (predicted == labels).sum().item()
            
            total_predictions += labels.size(0)
            
            # 更新进度条
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{correct_predictions/total_predictions:.4f}',
                'Mixup': 'ON' if (mixup_augmenter is not None and lambda_ != 1.0) else 'OF'
            })
        
        avg_loss = total_loss / len(train_loader)
        accuracy = correct_predictions / total_predictions
        
        return avg_loss, accuracy
    
    def validate(self, val_loader, criterion):
        """验证"""
        self.model.eval()
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0
        all_predictions = []
        all_labels = []
        
        with torch.no_grad():
            for sequences, labels, lengths in tqdm(val_loader, desc="验证中", disable=True):
                sequences = sequences.to(self.device)
                labels = labels.to(self.device)
                
                outputs = self.model(sequences)
                loss = criterion(outputs, labels)
                
                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_predictions += labels.size(0)
                
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_loss = total_loss / len(val_loader)
        accuracy = correct_predictions / total_predictions
        
        return avg_loss, accuracy, all_predictions, all_labels
    
    def train(self, train_loader, val_loader, num_epochs=50, learning_rate=0.001,
              weight_decay=1e-5, patience=10, save_path='best_model.pth', mixup_augmenter=None):
        """完整训练流程"""
        
        # 优化器和损失函数
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        criterion = self.criterion
        
        # 学习率调度器
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=patience//2
        )
        
        # 早停
        best_val_score = -float('inf')
        epochs_without_improvement = 0
        
        print(f"开始训练，设备: {self.device}")
        print(f"模型参数量: {sum(p.numel() for p in self.model.parameters()):,}")
        
        # 记录当前学习率用于检测变化
        current_lr = learning_rate
        
        for epoch in range(num_epochs):
            # 训练
            train_loss, train_acc = self.train_epoch(train_loader, optimizer, criterion, mixup_augmenter)
            
            # 验证
            val_loss, val_acc, val_preds, val_labels = self.validate(val_loader, criterion)
            val_preds = [self.class_names[pred] for pred in val_preds]
            val_labels = [self.class_names[label] for label in val_labels]
            val_score = get_competition_score(val_labels, val_preds)
            
            # 记录历史
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.train_accuracies.append(train_acc)
            self.val_accuracies.append(val_acc)
            self.val_scores.append(val_score)
            
            # 更新学习率
            old_lr = optimizer.param_groups[0]['lr']
            scheduler.step(val_score)
            new_lr = optimizer.param_groups[0]['lr']
            
            # 手动记录学习率变化
            # if new_lr != old_lr:
            #     print(f"学习率从 {old_lr:.6f} 降低到 {new_lr:.6f}")

            if epoch % 10 == 0:
                print(f"\nEpoch {epoch+1}/{num_epochs}")
                print("-" * 50)
                print(f"训练 - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
                print(f"验证 - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, Score: {val_score:.4f}")
                print(f"当前学习率: {new_lr:.6f}")
            
            # 早停检查
            if val_score > best_val_score:
                best_val_score = val_score
                epochs_without_improvement = 0
                # 保存最佳模型
                torch.save({
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'epoch': epoch,
                    'val_loss': val_loss,
                    'val_acc': val_acc,
                    'val_score': val_score,
                    'model_config': {                        
                        'imu_dim': self.model.imu_dim,
                        'tof_dim': self.model.tof_dim,
                        'n_classes': self.model.n_classes,
                        'weight_decay': self.model.weight_decay
                    }
                }, save_path)
                # print(f"保存最佳模型到 {save_path}")
            else:
                epochs_without_improvement += 1
                
            if epochs_without_improvement >= patience:
                print(f"早停：{patience} 个epoch无改善")
                break
        
        print("训练完成！")
        return self.train_losses, self.val_losses, self.train_accuracies, self.val_accuracies, self.val_scores
    
    def plot_training_history(self, save_path='training_history.png'):
        """绘制训练历史"""
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
        
        # 损失曲线
        ax1.plot(self.train_losses, label='train loss')
        ax1.plot(self.val_losses, label='val loss')
        ax1.set_title('train and val loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True)
        
        # 准确率曲线
        ax2.plot(self.train_accuracies, label='train accuracy')
        ax2.plot(self.val_accuracies, label='val accuracy')
        ax2.set_title('train and val accuracy')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy')
        ax2.legend()
        ax2.grid(True)

        # 分数曲线
        ax3.plot(self.val_scores, label='val score')
        ax3.set_title('val score')
        ax3.set_xlabel('Epoch')
        ax3.set_ylabel('Score')
        ax3.legend()
        ax3.grid(True)
        
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"训练历史图保存到 {save_path}")


def evaluate_model(model, test_loader, device, class_names=None):
    """评估模型"""
    model.eval()
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    with torch.no_grad():
        for sequences, labels, lengths in tqdm(test_loader, desc="评估中", disable=True):
            sequences = sequences.to(device)
            labels = labels.to(device)
            
            outputs = model(sequences)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    # 计算指标
    accuracy = accuracy_score(all_labels, all_predictions)
    score = get_competition_score(
        [class_names[label] for label in all_labels],
        [class_names[pred] for pred in all_predictions]
    )

    # 分类报告
    if class_names is None:
        class_names = [f'Class_{i}' for i in range(len(set(all_labels)))]
    
    # import pdb; pdb.set_trace()
    sub_class_names = sorted(set(all_labels + all_predictions))
    sub_class_names = [class_names[i] for i in sub_class_names]
    report = classification_report(all_labels, all_predictions, 
                                 target_names=sub_class_names, output_dict=True)
    
    print(f"测试准确率: {accuracy:.4f}")
    print(f"测试分数: {score:.4f}")
    print("\n分类报告:")
    print(classification_report(all_labels, all_predictions, target_names=sub_class_names))
    
    # 混淆矩阵
    cm = confusion_matrix(all_labels, all_predictions)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=sub_class_names, yticklabels=sub_class_names)
    plt.title('confusion matrix')
    plt.ylabel('true label')
    plt.xlabel('predicted label')
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return accuracy, score, report, all_predictions, all_probabilities

In [ ]:
def train_single_fold(sequences, labels, subjects, train_idx, val_idx,
                     fold_num, total_folds, device, num_epochs=50, 
                     learning_rate=0.001, patience=10, batch_size=16, label_smoothing=0.0,
                     use_mixup=False, mixup_params=None):
    """训练单个折"""
    print(f"\n{'='*60}")
    print(f"🔄 开始训练第 {fold_num}/{total_folds} 折")
    print(f"{'='*60}")
    
    # 创建数据加载器
    train_loader, val_loader, mixup_augmenter = create_fold_loaders(
        sequences, labels, train_idx, val_idx, batch_size=batch_size,
        use_mixup=use_mixup, mixup_params=mixup_params
    )
    
    print(f"📊 第{fold_num}折数据信息:")
    print(f"   训练样本: {len(train_loader.dataset)}")
    print(f"   验证样本: {len(val_loader.dataset)}")
    
    # 检查subject分布
    train_subjects_fold = [subjects[i] for i in train_idx]
    val_subjects_fold = [subjects[i] for i in val_idx]
    print(f"   训练subjects: {len(set(train_subjects_fold))}")
    print(f"   验证subjects: {len(set(val_subjects_fold))}")
    
    # 创建模型
    num_classes = len(LABEL_NAMES)
    model = TwoBranchModel(
        imu_dim=20,
        tof_dim=len(FEATURE_NAMES) - 20,
        n_classes=num_classes,
        weight_decay=3e-3
    )
    
    # 初始化权重
    if hasattr(model, 'init_weights'):
        model.init_weights()
    
    print(f"✅ 模型参数量: {sum(p.numel() for p in model.parameters()):,}")
    
    # 创建训练器
    trainer = Trainer(model, device, LABEL_NAMES, label_smoothing=label_smoothing,)
    
    # 训练参数
    model_save_path = f'{OUTPUT_DIR}/fold_{fold_num}_model.pth'
    
    # 训练
    print(f"🏋️ 开始第{fold_num}折训练...")
    train_losses, val_losses, train_accuracies, val_accuracies, val_scores = trainer.train(
        train_loader, val_loader,
        num_epochs=num_epochs,
        learning_rate=learning_rate,
        patience=patience,
        save_path=model_save_path,
        mixup_augmenter=mixup_augmenter
    )
    
    print(f"✅ 第{fold_num}折训练完成!")
    print(f"📈 最终训练准确率: {train_accuracies[-1]:.4f}")
    print(f"📊 最终验证准确率: {val_accuracies[-1]:.4f}")
    print(f"📊 最终验证分数: {val_scores[-1]:.4f}")

    # 绘制训练历史
    history_plot_path = f'{OUTPUT_DIR}/fold_{fold_num}_training_history.png'
    trainer.plot_training_history(history_plot_path)
    
    # 加载最佳模型进行评估
    checkpoint = torch.load(model_save_path, map_location=device, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # 最终评估
    accuracy, score, report, predictions, probabilities = evaluate_model(
        model, val_loader, device, LABEL_NAMES
    )
    
    # 保存该折的结果
    fold_results = {
        'fold': fold_num,
        'model_path': model_save_path,
        'final_train_acc': train_accuracies[-1],
        'final_val_acc': val_accuracies[-1],
        'final_val_score': val_scores[-1],
        'test_accuracy': accuracy,
        'test_score': score,
        'num_epochs': len(train_losses),
        'train_subjects': len(set(train_subjects_fold)),
        'val_subjects': len(set(val_subjects_fold))
    }
    
    print(f"🎯 第{fold_num}折最终评估:")
    print(f"   测试准确率: {accuracy:.4f}")
    print(f"   测试分数: {score:.4f}")
    
    return fold_results


def run_kfold_training(train_df, n_folds=5, **kwargs):
    """运行K折交叉验证训练"""
    print("🚀 使用K折交叉验证进行LSTM序列分类训练")
    print("=" * 60)
    
    # 设置设备
    device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    print(f"🧠 使用设备: {device}")
    
    # 加载数据
    print("📊 加载训练数据...")
    # train_df = pd.read_json(train_path, lines=True)
    sequences = train_df['sequence'].tolist()
    labels = train_df['label'].tolist()
    subjects = train_df['subject'].tolist()
    
    num_classes = len(LABEL_NAMES)
    print(f"\n📋 数据信息:")
    print(f"   总样本数: {len(sequences)}")
    print(f"   类别数量: {num_classes}")
    print(f"   总subjects数: {len(set(subjects))}")
    
    # 显示类别信息
    print(f"\n🏷️  类别示例:")
    for i, class_name in enumerate(LABEL_NAMES[:8]):
        print(f"   {i}: {class_name}")
    if len(LABEL_NAMES) > 8:
        print(f"   ... 还有 {len(LABEL_NAMES) - 8} 个类别")
    
    # 创建K折分割
    print(f"\n📂 创建{n_folds}折交叉验证分割...")
    splits = create_kfold_splits(sequences, labels, subjects, n_splits=n_folds, random_state=42)
    
    # K折训练
    fold_results = []
    for fold_num, (train_idx, val_idx) in enumerate(splits, 1):
        fold_result = train_single_fold(
            sequences, labels, subjects, train_idx, val_idx,
            fold_num, n_folds, device, **kwargs
        )
        fold_results.append(fold_result)
    
    # 计算总体统计
    print(f"\n{'='*60}")
    print(f"🎉 {n_folds}折交叉验证完成!")
    print(f"{'='*60}")
    
    val_accuracies = [r['final_val_acc'] for r in fold_results]
    val_scores = [r['final_val_score'] for r in fold_results]
    test_accuracies = [r['test_accuracy'] for r in fold_results]
    test_scores = [r['test_score'] for r in fold_results]
    
    print(f"\n📊 K折交叉验证统计:")
    print(f"   验证准确率: {np.mean(val_accuracies):.4f} ± {np.std(val_accuracies):.4f}")
    print(f"   验证分数: {np.mean(val_scores):.4f} ± {np.std(val_scores):.4f}")
    print(f"   测试准确率: {np.mean(test_accuracies):.4f} ± {np.std(test_accuracies):.4f}")
    print(f"   测试分数: {np.mean(test_scores):.4f} ± {np.std(test_scores):.4f}")
    
    # 显示每折结果
    print(f"\n📋 各折详细结果:")
    for i, result in enumerate(fold_results):
        print(f"   Fold {i+1}: Val Acc={result['final_val_acc']:.4f}, "
              f"Val Score={result['final_val_score']:.4f}, "
              f"Test Acc={result['test_accuracy']:.4f}, "
              f"Test Score={result['test_score']:.4f}")
    
    # 保存K折结果
    kfold_results = {
        'n_folds': n_folds,
        'fold_results': fold_results,
        'summary': {
            'val_acc_mean': np.mean(val_accuracies),
            'val_acc_std': np.std(val_accuracies),
            'val_score_mean': np.mean(val_scores),
            'val_score_std': np.std(val_scores),
            'test_acc_mean': np.mean(test_accuracies),
            'test_acc_std': np.std(test_accuracies),
            'test_score_mean': np.mean(test_scores),
            'test_score_std': np.std(test_scores)
        },
    }
    
    with open(f'{OUTPUT_DIR}/kfold_results.json', 'w', encoding='utf-8') as f:
        json.dump(kfold_results, f, ensure_ascii=False, indent=2)

    print(f"💾 K折结果已保存: kfold_results.json")
    
    return kfold_results

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

# 训练参数
training_params = {
    'num_epochs': 1 if DEBUG else 100,
    'learning_rate': 0.001,
    'patience': 20,
    'batch_size': 12,
    'label_smoothing': 0.1,
    'use_mixup': True,                  # 启用Mixup数据增强
    'mixup_params': {                   # Mixup参数
        'alpha': 0.2,                   # Beta分布参数，控制混合程度
        'prob': 0.5                     # 应用Mixup的概率
    }
}

if DEBUG:
    agg_train_df = agg_train_df.head(2000)

# K折交叉验证训练
n_folds = 5
print("🔄 开始K折交叉验证训练")
print(f"   - Use Mixup: {training_params['use_mixup']}")
if training_params['use_mixup']:
    print(f"   - Mixup Alpha: {training_params['mixup_params']['alpha']}")
    print(f"   - Mixup Probability: {training_params['mixup_params']['prob']}")

if TRAIN:
    _ = run_kfold_training(agg_train_df, n_folds=n_folds, **training_params)

In [ ]:
ls saved_models/

## Inference

In [ ]:
import os

def get_lastest_saved_models():
    latest = 1
    for version in os.listdir('/kaggle/input/cmi-imu-only-lstm-trained-output/pytorch/default/'):
        try:
            version = int(version)
        except:
            continue

        if version > latest:
            latest = version

    return f'/kaggle/input/cmi-imu-only-lstm-trained-output/pytorch/default/{latest}/saved_models/'

In [ ]:
SAVED_MODELS = './saved_models/' if TRAIN else get_lastest_saved_models()

In [ ]:
import torch
import numpy as np
import pickle
import glob
import os
from collections import Counter
from typing import List, Dict
import polars as pl

import joblib

In [ ]:
class EnsemblePredictor:
    """集成预测器 - 使用多个fold模型投票预测"""
    
    def __init__(self, models_dir, device='cpu'):
        """
        初始化集成预测器
        
        Args:
            device: 计算设备
        """
        self.device = device
        self.models = []
        self._load_models(models_dir)
    
    def _load_models(self, models_dir):
        """加载所有fold的模型"""
        # 查找所有fold模型文件
        model_files = sorted(glob.glob(f"{models_dir}/fold_*_model.pth"))
        print(f"📁 找到 {len(model_files)} 个fold模型")
        
        # 加载每个fold的模型
        for model_file in model_files:
            checkpoint = torch.load(model_file, map_location=self.device, weights_only=False)
            config = checkpoint['model_config']
            
            model = TwoBranchModel(**config)
            
            model.load_state_dict(checkpoint['model_state_dict'])
            model.to(self.device)
            model.eval()
            self.models.append(model)
            
            print(f"✅ 加载 {model_file}")
        
    def predict(self, sequence: List[Dict]) -> str:
        """
        预测单个序列的标签
        
        Args:
            sequence: 输入序列，格式与训练数据相同
        
        Returns:
            predicted_label: 预测的文本标签
        """
        predictions = []
        
        # 使用每个fold的模型进行预测
        for model in self.models:
            # 使用SequenceDataset处理序列
            dataset = SequenceDataset([sequence], [0], is_training=False, use_augmentation=False)
            
            # 获取处理后的序列
            processed_sequence = torch.FloatTensor(dataset.processed_sequences[0]).unsqueeze(0).to(self.device)
            
            # 预测
            with torch.no_grad():
                outputs = model(processed_sequence)
                predicted_class = torch.argmax(outputs, dim=1).item()
                predictions.append(predicted_class)
        
        # 投票决定最终预测结果
        most_common_prediction = Counter(predictions).most_common(1)[0][0]
        
        # 转换为文本标签
        predicted_label = LABEL_NAMES[most_common_prediction]
        
        return predicted_label

In [ ]:
feature_scaler = joblib.load(f'{SAVED_MODELS}/feature_scaler.joblib')

In [ ]:
predictor = EnsemblePredictor(SAVED_MODELS, device='cuda')

In [ ]:
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    sequence = sequence.to_pandas()
    demographics = demographics.to_pandas()

    sequence = pd.merge(
        sequence,
        demographics,
        on='subject',
        how='left'
    )

    sequence = feature_engineering(sequence)

    sequence[NUMERICAL_FEATURES] = feature_scaler.transform(sequence[NUMERICAL_FEATURES])
    
    sequence = sequence.groupby(['sequence_id', 'subject']).apply(
        lambda df: df[FEATURE_NAMES].to_dict(orient='records'),
        include_groups=False,
    )
    
    sequence = sequence.iloc[0]

    # predict
    predicted_label = predictor.predict(sequence)

    return predicted_label

## API

In [ ]:
import kaggle_evaluation.cmi_inference_server
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
        )
    )